In [19]:
from typing import Mapping, Any,Optional,List
from collections import defaultdict
import re
import numpy as np
import textworld
import textworld.gym
from textworld import EnvInfos
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Random Agent


In [20]:
class RandomAgent(textworld.gym.Agent):
    def __init__(self,seed=10):
        self.seed = seed
        self.rng = np.random.RandomState(self.seed)
    
    @property
    def infos_to_request(self) -> textworld.EnvInfos:
        return textworld.EnvInfos(admissible_commands = True)
    
    def act(self,obs:str,score:int,done:bool,infos:Mapping[str,Any]) -> str:
        return self.rng.choice(infos['admissible_commands'])

## Play Function

In [21]:
import os
from glob import glob
import gym

def play(agent,path,max_step=100,nb_episodes=10,verbose=True):
    infos_to_request = agent.infos_to_request
    infos_to_request.max_score = True
    
    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path,'*.ulx'))
    
    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=infos_to_request,
                                          max_episode_steps=max_step)
    env = gym.make(env_id)
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path),end="")
        else:
            print(os.path.basename(path),end="")
    
    avg_moves, avg_scores, avg_norm_scores = [],[],[]
    for no_episode in range(nb_episodes):
        obs, infos = env.reset() # To start new Episode
        score = 0
        done = False
        nb_moves = 0
        while not done:
            command = agent.act(obs,score,done,infos)
            obs,score,done,info = env.step(command)
            nb_moves+=1
            
        agent.act(obs,score,done,infos)
        
        if verbose:
            print(".",end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score/infos["max_score"])
    
    env.close()
    msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
    
    if verbose:
        if os.path.isdir(path):
            print(msg.format(np.mean(avg_moves),np.mean(avg_norm_scores),1))
        else:
            print(msg.format(np.mean(avg_moves),np.mean(avg_scores),infos["max_score"]))

## Testing the Random Agent

In [22]:
play(RandomAgent(),"./games/rewardsDense_goalDetailed.ulx")
play(RandomAgent(), "./games/rewardsBalanced_goalDetailed.ulx")
play(RandomAgent(), "./games/rewardsSparse_goalDetailed.ulx")

rewardsDense_goalDetailed.ulx..........  	avg. steps: 100.0; avg. score:  1.0 / 10.
rewardsBalanced_goalDetailed.ulx..........  	avg. steps: 100.0; avg. score:  0.0 / 4.
rewardsSparse_goalDetailed.ulx..........  	avg. steps: 100.0; avg. score:  0.0 / 1.


## Scorer

In [23]:
class CommandScorer(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(CommandScorer,self).__init__()
        torch.manual_seed(1)
        self.embedding = nn.Embedding(input_size,hidden_size)
        self.encoder_gru = nn.GRU(hidden_size,hidden_size)
        self.cmd_encoder_gru = nn.GRU(hidden_size,hidden_size)
        self.state_gru = nn.GRU(hidden_size,hidden_size)
        self.hidden_size = hidden_size
        self.state_hidden = torch.zeros(1, 1, hidden_size, device=device)
        self.critic = nn.Linear(hidden_size,1)
        self.att_cmd = nn.Linear(hidden_size*2,1)
    
    def forward(self,obs,commands,*kwargs):
        input_length = obs.size(0)
        batch_size = obs.size(1)
        nb_cmds = commands.size(1)
        
        embedded = self.embedding(obs)
        encoder_output, encoder_hidden = self.encoder_gru(embedded)
        state_output, state_hidden = self.state_gru(encoder_hidden,self.state_hidden)
        self.state_hidden = state_hidden
        value = self.critic(state_output)
        
        #Attention network over the commands
        cmds_embedding = self.embedding.forward(commands)
        _,cmds_encoding_last_states = self.cmd_encoder_gru.forward(cmds_embedding) # 1 x cmds x hidden 
        
        #Same observed state for all commands
        cmd_selector_input = torch.stack([state_hidden]*nb_cmds,2) # 1 x batch x cmds x hidden
        
        #Same command choices for whole batch
        cmds_encoding_last_states = torch.stack([cmds_encoding_last_states]*batch_size,1) # 1 x batch x cmds x hidden
        
        # Concatenate the observed state and command encodings
        cmd_selector_input = torch.cat([cmd_selector_input,cmds_encoding_last_states],dim=-1)
        
        #Compute on score per command
        scores = F.relu(self.att_cmd(cmd_selector_input)).squeeze(-1) # 1x batch x cmds
        
        probs = F.softmax(scores,dim=2) # 1 x batch x cmds
        index = probs[0].multinomial(num_samples=1).unsqueeze(0) # 1 x batch x index
        return scores,index,value
    
    def reset_hidden(self,batch_size):
        self.state_hidden = torch.zeros(1,batch_size,self.hidden_size,device=device)

## Neural Agent (with PyTorch)

In [24]:
MAX_VOCAB_SIZE = 1000
UPDATE_FREQUENCY = 10
LOG_FREQUENCY = 1000
GAMMA = 0.9

In [25]:
class NeuralAgent:
    
    def __init__(self) -> None:
        self._initialized = False
        self._episode_has_started = False
        self.id2word = ["<PAD>","<UNK>"]
        self.word2id = {w:i for i,w in enumerate(self.id2word)}
        self.MAX_VOCAB_SIZE = 1000
        self.UPDATE_FREQUENCY = 10
        self.LOG_FREQUENCY = 1000
        self.GAMMA = 0.9
        self.model = CommandScorer(input_size = MAX_VOCAB_SIZE,hidden_size = 128)
        self.optimizer = optim.Adam(self.model.parameters(),0.00003)
        
        self.mode = "test"
    
    def train(self):
        self.mode = "train"
        self.stats = {"max":defaultdict(list),"mean":defaultdict(list)}
        self.transitions = []
        self.model.reset_hidden(1)
        self.last_score = 0 
        self.no_train_step = 0
        
    def test(self):
        self.mode = "test"
        self.model.reset_hidden(1)
    
    @property
    def infos_to_request(self) -> EnvInfos:
        return EnvInfos(description=True,inventory=True,admissible_commands=True,won=True,lost=True)
    
    def get_word_id(self,word):
        if word not in self.word2id:
            if len(self.word2id) >= self.MAX_VOCAB_SIZE:
                return self.word2id["<UNK>"]
            
            self.id2word.append(word)
            self.word2id[word] = len(self.word2id)
        return self.word2id[word]
    
    def _tokenize(self,text):
        # To strip out all non-alphabetic characters
        text = re.sub("[^a-zA-Z0-9\- ]", " ", text)
        word_ids = list(map(self.get_word_id,text.split()))
        return word_ids
    
    def _process(self,texts):
        texts = list(map(self._tokenize,texts))
        max_len = max(len(l) for l in texts)
        padded = np.ones((len(texts),max_len)) * self.word2id["<PAD>"]
        
        for i,text in enumerate(texts):
            padded[i,:len(text)] = text
        
        padded_tensor = torch.from_numpy(padded).type(torch.long).to(device)
        padded_tensor = padded_tensor.permute(1,0) # Batch x Seq => Seq x Batch
        return padded_tensor
        
    def _discount_rewards(self,last_values):
        returns, advantages = [], []
        R = last_values.data
        for t in reversed(range(len(self.transitions))):
            rewards, _, _, values = self.transitions[t]
            R = rewards + self.GAMMA * R
            adv = R - values
            returns.append(R)
            advantages.append(adv)
        
        return returns[::-1], advantages[::-1]
    
    def act(self,obs:str,score:int,done:bool,infos : Mapping[str,Any]) -> Optional[str]:
        
        # Build Agents observation : feedback + look + inventory
        input_ = "{}\n{}\n{}".format(obs,infos['description'],infos['inventory'])
        
        #Tokenize and pad the input and the commands to choose from
        input_tensor = self._process([input_])
        commands_tensor = self._process(infos["admissible_commands"])
        
        # Get our next action and value prediction
        outputs, indexes, values = self.model(input_tensor,commands_tensor)
        action = infos["admissible_commands"][indexes[0]]
        
        if self.mode == "test":
            if done:
                self.model.reset_hidden(1)
            return action
        
        self.no_train_step += 1
        
        if self.transitions:
            reward = score - self.last_score # Reward is the gain or loss in the game.
            self.last_score = score
            if infos["won"]:
                reward += 100
            if infos["lost"]:
                reward -= 100
                
            self.transitions[-1][0] = reward  # Update reward information.
        
        self.stats["max"]["score"].append(score)
        if self.no_train_step % self.UPDATE_FREQUENCY == 0:
            # Update model
            returns, advantages = self._discount_rewards(values)
            
            loss = 0
            for transition, ret, advantage in zip(self.transitions, returns, advantages):
                reward, indexes_, outputs_, values_ = transition
                
                advantage        = advantage.detach() # Block gradients flow here.
                probs            = F.softmax(outputs_, dim=2)
                log_probs        = torch.log(probs)
                log_action_probs = log_probs.gather(2, indexes_)
                policy_loss      = (-log_action_probs * advantage).sum()
                value_loss       = (.5 * (values_ - ret) ** 2.).sum()
                entropy     = (-probs * log_probs).sum()
                loss += policy_loss + 0.5 * value_loss - 0.1 * entropy
                
                self.stats["mean"]["reward"].append(reward)
                self.stats["mean"]["policy"].append(policy_loss.item())
                self.stats["mean"]["value"].append(value_loss.item())
                self.stats["mean"]["entropy"].append(entropy.item())
                self.stats["mean"]["confidence"].append(torch.exp(log_action_probs).item())
            
            if self.no_train_step % self.LOG_FREQUENCY == 0:
                msg = "{}. ".format(self.no_train_step)
                msg += "  ".join("{}: {:.3f}".format(k, np.mean(v)) for k, v in self.stats["mean"].items())
                msg += "  " + "  ".join("{}: {}".format(k, np.max(v)) for k, v in self.stats["max"].items())
                msg += "  vocab: {}".format(len(self.id2word))
                print(msg)
                self.stats = {"max": defaultdict(list), "mean": defaultdict(list)}
            
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(),40)
            self.optimizer.step()
            self.optimizer.zero_grad()
            
            self.transitions = []
            self.model.reset_hidden(1)
        else:
            self.transitions.append([None, indexes, outputs, values])
            
        if done:
            self.last_score = 0  # Will be starting a new episode. Reset the last score.
        
        return action

## Training the neural agent

In [26]:
agent = NeuralAgent()
play(agent, "./games/rewardsDense_goalDetailed.ulx")

rewardsDense_goalDetailed.ulx..........  	avg. steps: 100.0; avg. score:  1.0 / 10.


In [27]:
from time import time
agent = NeuralAgent()

print("Training")
agent.train()
starttime = time()
play(agent,"./games/rewardsDense_goalDetailed.ulx", nb_episodes=500, verbose=False)
print("Trained in {:.2f} secs".format(time() - starttime))

Training
100. reward: 0.011  policy: 0.060  value: 0.011  entropy: 2.079  confidence: 0.126  score: 1  vocab: 145
200. reward: 0.011  policy: 0.066  value: 0.017  entropy: 2.079  confidence: 0.125  score: 1  vocab: 145
300. reward: 0.011  policy: 0.084  value: 0.022  entropy: 2.079  confidence: 0.125  score: 1  vocab: 145
400. reward: 0.011  policy: 0.028  value: 0.017  entropy: 2.079  confidence: 0.125  score: 1  vocab: 145
500. reward: 0.011  policy: 0.033  value: 0.019  entropy: 2.079  confidence: 0.125  score: 1  vocab: 145
600. reward: 0.011  policy: -0.018  value: 0.011  entropy: 2.079  confidence: 0.125  score: 1  vocab: 145
700. reward: 0.011  policy: -0.012  value: 0.011  entropy: 2.079  confidence: 0.125  score: 1  vocab: 145
800. reward: 0.011  policy: -0.022  value: 0.006  entropy: 2.079  confidence: 0.125  score: 1  vocab: 145
900. reward: 0.011  policy: 0.008  value: 0.010  entropy: 2.079  confidence: 0.125  score: 1  vocab: 145
1000. reward: 0.011  policy: 0.059  value: 

KeyboardInterrupt: 

In [ ]:
# We report the score and steps averaged over 10 playthroughs.
agent.test()
play(agent, "./games/rewardsDense_goalDetailed.ulx")  # Dense rewards game.